# 🎨 Multimodalna klasifikacija umetničkih stilova

In [ ]:
%pip install pandas numpy opencv-python tensorflow scikit-learn datasets matplotlib seaborn tabulate

Cilj ovog projekta je klasifikacija umetničkih slika prema stilu pomoću **konvolucionih neuronskih mreža (CNN)** i **multimodalnog pristupa** koji kombinuje slike i metapodatke (umetnik, godina, žanr).


In [ ]:

import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Concatenate, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns

## Parametri eksperimenta

Definišu se osnovni parametri kao što su veličina slike, broj epoha i broj uzoraka koji se koristi iz WikiArt skupa podataka.

In [ ]:
MAX_SAMPLES = 20000
TOP_N_STYLES = 50 
TOP_N_ARTISTS = 40
TOP_N_GENRES = 40 
IMG_SIZE = 96
BATCH_SIZE = 32
NUM_EPOCHS = 20
early_stop = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

## Učitavanje i podela podataka

Koristi se **WikiArt** dataset. Slike se preprocesiraju na $96 \times 96$, dok se metapodaci (godina, top $40$ umetnika/žanrova) transformišu u numeričke vektore za fuziju. Zadržano je $50$ najčešćih stilova za klasifikaciju. Podaci su podeljeni na trening, validacioni i test skup.

In [ ]:
from artstyles import load_data 

(X_train_img, X_val_img, X_test_img,
 y_train_cat, y_val_cat, y_test_cat,
 y_test_enc, le, train_meta, val_meta, test_meta, NUM_CLASSES) = load_data()

print(f"Trening primeri: {len(X_train_img)}")
print(f"Validacioni primeri {len(X_val_img)}")
print(f"Test primeri: {len(X_test_img)}")

In [ ]:
def plot_history(history, model_name):
    plt.figure(figsize=(15, 5))
    acc_key = 'accuracy' if 'accuracy' in history.history else 'acc'
    val_acc_key = 'val_accuracy' if 'val_accuracy' in history.history else 'val_acc'

    # Loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Trening Loss')
    plt.plot(history.history['val_loss'], label='Validacioni Loss')
    plt.title(f'Loss - {model_name}')
    plt.xlabel('Epoha')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history[acc_key], label='Trening Accuracy')
    plt.plot(history.history[val_acc_key], label='Validacioni Accuracy')
    plt.title(f'Accuracy - {model_name}')
    plt.xlabel('Epoha')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

## Treniranje modela

Treniraju se dva modela:
1. **Samo CNN (Vizuelni model):** Koristi samo slike
2. **Multimodalni model (Fuzija):** Kombinuje slike i metapodatke

Za kontrolu *overfittinga* koristi se Augmentacija slika i rani prekid (`EarlyStopping` sa `patience=6`).

In [ ]:
from artstyles import train_models

cnn_model, history_cnn, multi_model, history_multi = train_models(
    X_train_img, y_train_cat, X_val_img, y_val_cat,
    train_meta, val_meta, NUM_CLASSES, BATCH_SIZE, NUM_EPOCHS, early_stop
)

In [ ]:
plot_history(history_cnn, "Samo CNN Model")
plot_history(history_multi, "Multimodalni Model (Slika + Metapodaci)")

## Evaluacija i prikaz rezultata

- Izračunava se tačnost i F1-score za oba modela.
- Prikazuje se confusion matrix za bolji model.

In [ ]:
from artstyles import evaluate_models

results = evaluate_models(
    cnn_model, multi_model,
    X_test_img, test_meta,
    y_test_enc, y_test_cat, le
)

results

# ✅ Zaključak

Glavni nalaz ovog eksperimenta je **dramatična superiornost multimodalnog modela** u klasifikaciji umetničkog stila.

1. **Samo slike:** Model koji koristi **samo vizuelne podatke (CNN)**, nakon svih optimizacija, postigao je tačnost od samo **$\approx 21.5\%$**. Ova niska tačnost (na 50 klasa) ukazuje da su podaci niske rezolucije **nedovoljni** za hvatanje suptilnih vizuelnih karakteristika stila.

2. **Metapodaci i slike:** Model koji kombinuje slike i metapodatke (**CNN + Metapodaci**) postigao je tačnost od **$\approx 40\%$**. To predstavlja poboljšanje performansi od približno **$19$%**.

3. **Rezultat:** Ovaj rezultat snažno dokazuje da su informacije o **umetniku, datumu nastanka i žanru** od kritične važnosti i da su ključne za precizno određivanje stila, efektivno **kompenzujući** nedostatak finih vizuelnih detalja.